<a href="https://colab.research.google.com/github/LeonardoDrLourenco/IA2_AtividadeAula5_QLearns/blob/main/IA2_AtividadeAula5_QLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fazer uma pesquisa sobre Q-Learning com redes neurais. Quais são as principais diferenças comparando com o modelo matemático apresentado?

O Q-Learning é uma técnica usada para ensinar um agente a tomar decisões com base nas suas experiências. Ele funciona armazenando uma tabela que diz qual é a melhor ação a tomar em cada situação. O problema é que essa tabela só funciona bem para problemas pequenos, porque quando o ambiente tem muitas opções, essa tabela fica gigantesca e difícil de lidar.

Por outro lado, as redes neurais são ótimas para aprender padrões em grandes quantidades de dados. Elas podem substituir essa tabela do Q-Learning e fazer o agente tomar boas decisões mesmo em ambientes super complexos, como jogos de videogame ou problemas com muitas variáveis.

Quando juntamos as duas ideias, temos o Deep Q-Learning (DQN). Nesse caso, a rede neural aprende a calcular as melhores ações a serem tomadas, o que faz com que o agente consiga lidar com problemas muito maiores e mais complicados do que o Q-Learning clássico sozinho.


### Principais diferenças:

Representação dos estados e ações: No Q-Learning clássico, os estados e as ações são discretos e armazenados em uma tabela, enquanto no DQN, a rede neural aprende a mapear estados contínuos para valores Q.

Escalabilidade: O DQN consegue lidar com ambientes complexos onde seria inviável armazenar todos os pares estado-ação. No Q-Learning clássico, isso é limitado pelo tamanho da tabela Q.

Generalização: As redes neurais têm a capacidade de generalizar, o que significa que o agente pode aprender a tomar boas decisões mesmo em estados que não foram vistos diretamente durante o treinamento, algo difícil de se obter com a abordagem baseada em tabela.

Instabilidade: O uso de redes neurais pode tornar o treinamento mais instável e demorado, necessitando de técnicas adicionais, como o replay de experiência e o uso de redes-alvo para estabilizar o aprendizado.


### Algoritmo Exemplo

Esse código implementa um agente de aprendizado por reforço usando uma Rede Neural Profunda (DQN) para jogar o jogo CartPole (CartPole é um ambiente de aprendizado por reforço onde o objetivo é equilibrar um pêndulo invertido em um carrinho móvel). O agente tenta equilibrar um pêndulo em uma vara, aprendendo ao longo de várias tentativas. Ele utiliza uma memória de replay para armazenar experiências passadas e uma estratégia chamada epsilon-greedy para equilibrar a exploração de novas ações e a exploração de ações conhecidas. Com o tempo, o agente se torna mais habilidoso em manter o pêndulo equilibrado e maximizar suas recompensas.

In [3]:
!pip install gymnasium[classic_control]  # Instala o Gym (fork do OpenAI Gym)
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque

# Definindo a rede neural DQN
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Memória de Replay
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return np.vstack(states), actions, rewards, np.vstack(next_states), dones

# Função de ação com epsilon-greedy
def act(state, epsilon):
    if random.random() < epsilon:
        return env.action_space.sample()
    else:
        state = torch.FloatTensor(state).unsqueeze(0)
        q_values = agent(state)
        return np.argmax(q_values.detach().numpy())

# Parâmetros
state_size = 4  # Tamanho do vetor de estado (CartPole)
action_size = 2  # Número de ações (esquerda ou direita)
batch_size = 32
gamma = 0.99  # Fator de desconto
epsilon = 1.0  # Taxa de exploração inicial
epsilon_min = 0.01
epsilon_decay = 0.995
learning_rate = 0.001
target_update = 10  # Frequência de atualização da rede alvo

# Configuração do ambiente
env = gym.make('CartPole-v1')
agent = DQN(state_size, action_size)
target_net = DQN(state_size, action_size)  # Rede alvo
target_net.load_state_dict(agent.state_dict())
optimizer = optim.Adam(agent.parameters(), lr=learning_rate)
memory = ReplayBuffer(10000)

# Treinamento
episodes = 300
for episode in range(episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = act(state, epsilon)
        next_state, reward, done, _, _ = env.step(action)
        memory.add(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

        # Atualizar a rede neural
        if len(memory.buffer) >= batch_size:
            states, actions, rewards, next_states, dones = memory.sample(batch_size)
            states = torch.FloatTensor(states)
            next_states = torch.FloatTensor(next_states)
            rewards = torch.FloatTensor(rewards)
            dones = torch.FloatTensor(dones)

            q_values = agent(states).gather(1, torch.LongTensor(actions).unsqueeze(1)).squeeze(1)
            next_q_values = target_net(next_states).max(1)[0]
            targets = rewards + gamma * next_q_values * (1 - dones)

            loss = (q_values - targets).pow(2).mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Atualizar a rede alvo
        if episode % target_update == 0:
            target_net.load_state_dict(agent.state_dict())

    # Reduzir epsilon (exploração)
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    print(f'Episódio {episode}, Total de Recompensas: {total_reward}')


Episódio 0, Total de Recompensas: 25.0
Episódio 1, Total de Recompensas: 19.0
Episódio 2, Total de Recompensas: 10.0
Episódio 3, Total de Recompensas: 12.0
Episódio 4, Total de Recompensas: 11.0
Episódio 5, Total de Recompensas: 13.0
Episódio 6, Total de Recompensas: 28.0
Episódio 7, Total de Recompensas: 18.0
Episódio 8, Total de Recompensas: 10.0
Episódio 9, Total de Recompensas: 11.0
Episódio 10, Total de Recompensas: 36.0
Episódio 11, Total de Recompensas: 27.0
Episódio 12, Total de Recompensas: 38.0
Episódio 13, Total de Recompensas: 29.0
Episódio 14, Total de Recompensas: 44.0
Episódio 15, Total de Recompensas: 19.0
Episódio 16, Total de Recompensas: 17.0
Episódio 17, Total de Recompensas: 41.0
Episódio 18, Total de Recompensas: 14.0
Episódio 19, Total de Recompensas: 21.0
Episódio 20, Total de Recompensas: 15.0
Episódio 21, Total de Recompensas: 10.0
Episódio 22, Total de Recompensas: 18.0
Episódio 23, Total de Recompensas: 12.0
Episódio 24, Total de Recompensas: 15.0
Episódio 2

Os resultados mostram que o agente de aprendizado por reforço no CartPole está passando por altos e baixos nas recompensas. Embora algumas recompensas sejam bem baixas, parece que, no geral, ele está aprendendo e melhorando com o tempo.